In [29]:
import pandas as pd
import numpy as np

df = pd.read_csv('mcdonaldata.csv')

In [30]:
# Keep only items that belong to regular, desert, or gourmet menus
df = df[df["menu"].isin(["regular", "desert", "gourmet"])].copy()

In [31]:
# Eliminate unnecessary columns
df = df[["item", "servesize", "calories", "protien"]].copy()


In [32]:
# Rename columns for clarity
df = df.rename(columns={
    "servesize": "serving_size",
    "protien": "protein"
})

In [33]:
# Convert numeric columns from string to numeric
numeric_cols = ["calories", "protein"]

df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

In [34]:
# Strip whitespace before/after item names
df["item"] = df["item"].str.strip()

In [35]:
# drop rows with missing protein or calories
df = df.dropna(subset=["calories", "protein"])

In [36]:
# set nonpositive calories to NaN (can't take log or divide by 0)
df.loc[df["calories"] <= 0, "calories"] = np.nan
df = df.dropna(subset=["calories"])  # remove the bad rows

In [37]:
# Protein per calorie (cutting score)
df["cutting_score"] = df["protein"] / df["calories"]

# Bulking score (protein * log(calories))
df["bulking_score"] = df["protein"] * np.log(df["calories"])

In [38]:
print(df)

                                           item serving_size  calories  \
0                               McVeggie Burger         168     402.00   
1                           McAloo Tikki Burger         146     339.00   
2                        McSpicy Paneer Burger         199     652.00   
3                             Spicy Paneer Wrap         250     674.00   
4                           American Veg Burger         177     512.00   
5                              Veg Maharaja Mac          306    832.00   
6                      Green Chilli Aloo Naan p        132      356.00   
7                                    Pizza Puff          87     228.00   
8                             Mc chicken Burger         173     400.00   
9                          FILLET-O-FISH Burger         136     348.00   
10                      Mc Spicy Chicken Burger         186     451.00   
11                           Spicy Chicken Wrap         257     567.00   
12                         Chicken Mah

In [ ]:
# Top 5 cutting (ranked best → 5th)
top_cutting = (df.sort_values("cutting_score", ascending=False)
                 .loc[:, ["item", "calories", "protein", "cutting_score"]]
                 .head(5).reset_index(drop=True))

# Top 5 bulking (ranked best → 5th)
top_bulking = (df.sort_values("bulking_score", ascending=False)
                 .loc[:, ["item", "calories", "protein", "bulking_score"]]
                 .head(5).reset_index(drop=True))

# Insert rank column for top items
top_cutting.insert(0, "Rank", range(1, len(top_cutting) + 1))
top_bulking.insert(0, "Rank", range(1, len(top_bulking) + 1))


In [40]:
# Rounding for display
top_cutting["cutting_score"] = top_cutting["cutting_score"].round(3)
top_bulking["bulking_score"] = top_bulking["bulking_score"].round(2)

In [41]:
print("Top 5 for CUTTING (most protein per calorie):")
print(top_cutting.to_string(index=True))

print("\nTop 5 for BULKING (protein * log(calories)):")
print(top_bulking.to_string(index=True))

Top 5 for CUTTING (most protein per calorie):
                         item  calories  protein  cutting_score
0  McSpicy Fried Chicken 1 pc     248.0    17.33          0.070
1      3 piece Chicken Strips     246.0    15.26          0.062
2      2 piece Chicken Strips     164.0    10.17          0.062
3      5 piece Chicken Strips     411.0    25.43          0.062
4   4 piece Chicken McNuggets     169.0    10.03          0.059

Top 5 for BULKING (protein * log(calories)):
                                         item  calories  protein  bulking_score
0                        Chicken Maharaja Mac     689.0    34.00         222.20
1  Ghee Rice with Mc Spicy Fried Chicken 1 pc     720.0    26.91         177.05
2                            Veg Maharaja Mac     832.0    24.17         162.52
3                      5 piece Chicken Strips     411.0    25.43         153.05
4                          Spicy Chicken Wrap     567.0    23.74         150.52


In [42]:
# Save the top cutting and bulking items to CSV files
top_cutting.to_csv("top_cutting.csv", index=False)
top_bulking.to_csv("top_bulking.csv", index=False)
